<a href="https://colab.research.google.com/github/tommasoDR/AIF-23-Project/blob/main/notebooks/report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h2>ARTIFICIAL INTELLIGENCE FOUNDAMENTALS PROJECT 2023-24</h2>

<h4><span style="color:lightblue">Github link</span>: https://github.com/tommasoDR/AIF-23-Project</h4>


<h4><b>1. INTRODUCTION</b></h4>

This project is centered around developing intelligent algorithms for agents to navigate through partially observable environments with optimal reasoning. The primary focus involves implementing an advanced exploration algorithm, based on A*, to enable the agent to effectively address our objectives. The agent's goal is to navigate sensibly within randomly generated rooms. It has to identify specific objects to recognize different room types, locate the target object and reach it. We used NetHack as the foundational framework for this project, offering a rich and complex environment for the development and testing of intelligent exploration algorithms.<br>
In the following, we’re going to explain the algorithms, the heuristics and the optimizations that let the agent solve this task smartly on a randomly generated map in the MiniHack environment.


<h4><b>2. RELATED WORKS</b></h4>

For generating the random environment in which the agent performs our smart algorithms we used MiniHack and gym libraries.<br>
[MiniHack](https://minihack.readthedocs.io/en/latest/getting-started/interface.html) is a software tool tailored for AI experimentation. It provides a simplified version of NetHack Learning Environment (NLE) with reduced complexity. It offers a controlled environment for training and testing AI agents and algorithms.<br>
[Gym]( https://www.gymlibrary.dev/index.html), by OpenAI, is a toolkit for reinforcement learning that provides a standardized interface for various environments, including MiniHack. In this collaboration, Gym acts as a bridge, allowing AI agents to interact with MiniHack through its standardized interface. By initializing Gym and MiniHack in our Python environment, we  developed an AI agent that exploits an optimized searching algorithm to perform our task in a controlled environment. This setup enables efficient experimentation and algorithm refinement for AI tasks within MiniHack.<br>
For creating room shapes and for choosing objects we checked the [NetHackWiki](https://nethackwiki.com/wiki/Main_Page).
For room exploration we decided to implement a modified version of A* with some optimizations. For this, we exploited the [AIMA documentation](https://github.com/yanshengjia/ml-road/blob/master/resources/Artificial%20Intelligence%20-%20A%20Modern%20Approach%20(3rd%20Edition).pdf) while the heuristics and the optimizations are created by us to let A* perform well in a partially observable environment.


<br>
<h4><b>3. TASK </b></h4>

The main task is to identify the room type the agent is in, among "Dragon Cave", "Samurai Temple of Doom" and "Abandoned Gold Mines of the Orcs", each of which has a three target object, two cursed and one not.
The agent's ability to discern room types relies on clue objects. These objects are placed within each room based on the probability of finding them in that specific room type. The collective probabilities of these objects to be inside a room always sum to one.
When the agent recognizes the type of room, exceeding a confidence threshold of the probability of being in that room, it consequently understands which target object is safe, reaches and gets it.<br><br>

 
<h4> 3.1 Setup  </h4>

The room in which the agent performs the experiment is randomly generated with a custom Python code created by us. The generator code randomly picks a room shape from different shapes inside .des files and a room type, then adds clue objects according to the probability of the room type and finally inserts all the target objects in random positions. 
We can easily add different room shapes by adding the .des file of the new room.  

<br>
<h4>3.2 Exploration</h4>

To solve the main task, the agent performs an exhaustive map exploration approach, employing the modified A* algorithm. At the beginning of the exploration the agent only knows the shape of the map without knowing its type or any object that is inside. <br>
At each step of the iteration the agent establishes the nearest unvisited cell to be reached, according to the chosen distance. It then moves along the path returned by A*, marking all cells in the path and their adjacent cells (i.e., the cells in its field of view) as visited. In each cell viewed by the agent, a check is made for the presence of objects, represented by symbols. Bayes' formula (1) is used to calculate and update the probabilities of being in each room, that are saved within a list. The updates of the probabilities are performed using the percentages calculated at the previous iterations and the percentages of finding the observed object in each room. Such exploration continues until the agent runs out of explorable cells or gains sufficient confidence about the type of room he is in. In both cases the agent heads to the target object of the most probable room type, concluding the execution of the exploration process. <br><br>

$$
\begin{equation}
\mathbf{P}\bigl(room \big| object \bigr) = \frac{\mathbf{P}\bigl(object\ \big|\ room \bigr) \cdot \mathbf{P}\bigl(room \bigr)}{\mathbf{P}\bigl(object \bigr)}
\tag{1}
\end{equation}
$$



**This is a demo that can be run on Google Colab**

In [ ]:
import warnings
import os
import sys

warnings.filterwarnings('ignore')
current_dir = %pwd

parent_dir = os.path.abspath(os.path.join(current_dir, '..'))
sys.path.append(parent_dir)

!git clone https://github.com/tommasoDR/AIF-23-Project.git
os.chdir("AIF-23-Project")
!pip install -r requirements.txt


In [ ]:
from src.explore_room import *

env, goals_info = generate_env()

guessed_room, _, _, _ = exhaustive_exploration(env.reset(), env)

if goals_info[guessed_room][3] == 'uncursed':
    print("Win")
else:
    print("Lose")

<br>
<h4>3.3 Find a new target during the exploration</h4>

As just mentioned, at each iteration of the exploration phase a new target is defined. The target represents the next cell that the agent will have to reach. All cells that the agent has yet to explore are considered candidates to become the new target. For each of these candidate cells, a heuristic function is calculated based on the current position of the agent. The cell with the best value of the heuristic becomes the new target.<br>
The heuristic developed by us for target selection is the TFFFM. This heuristic computes the Manhattan distance between the agent's position and the target and then adds a penalty term for each wall between them. In this way, we favor target cells that are as close as possible to the agent's position while penalizing paths that involve traversing walls.<br><br>


<h4>3.4 Conditioned map</h4>

When a target cell is chosen, the A* algorithm is invoked to find the shortest path between the agent position and the target. Instead of the original game map, A* operates on a pre-conditioned map. Pre-conditioning consists of replacing cells adjacent to the walls with "fake walls", ensuring that these changes do not block any path. Thanks to this, A* algorithm is forced to return a path always one cell away from the walls of the original game map. Since the agent can look at adjacent cells, moving to a cell next to a real wall would be unnecessary. Therefore, the conditioned map allows visiting the entire area while reducing the number of steps to be made, improving both efficiency and effectiveness.<br><br>


<h4>3.5 A* optimization</h4>

During exploration, there are cases where calling A* is unnecessary since the path is simply a straight line between the two cells (agent position and target). These instances can be easily detected by checking the coordinates of the two cells. When these cases are identified, A* is not executed; instead, the algorithm simply returns the path that corresponds to a straight line from agent position to target.
This optimization reduces the number of calls to A*, thereby improving the algorithm's performance in time and space.

<br>
<h4><b>4. ASSESSMENTS</b></h4>

Mainly, four different tests have been performed to evaluate the performance and correctness of our improvements. All the tests were run on the map with the most complex pattern. All results that will be shown are taken as average on 1000 runs, starting from a random location in the chosen map.
<br><br>

<h4>4.1 First Test</h4>

The first test regards the comparison of the number of steps taken by the agent to explore the entire map when using different distances and using or not the preconditioning of the map in the customized A* implementation. 
As we can see from the graph below, the number of steps taken by the agent is better when using preconditioning. Comparing the distances and fixing the preconditioning, it is easy to see that the TFFFM distance is better than the Manhattan distance for this problem; still, the difference is not too evident. Instead, fixing the distance and varying the preconditioning shows a much more relevant difference in the number of steps. This means that the preconditioning is very suitable for solving this task and results in a more clever exploration of the map. Thus, we can state that the main factor in decreasing the number of steps taken in an exhaustive exploration of the map is the latter.
<br><br>


<div style="display: flex; justify-content: space-between;">
  <figure>
    <img src="https://drive.google.com/uc?id=1rp7iGSW-E-lC2ol1TpIiZUlSt7DmqNLh" width="65%">
    <figcaption>
      Average number of steps over 1000 runs
    </figcaption>
  </figure>
</div>

<br><br>

<h4>4.2 Second Test</h4>

For the second test, we evaluated on average how many times the agent steps over the cells in one-thousand explorations over the map. We considered the comparison between the executions with our optimizations and without them, fixing the probability threshold to 100% to let the exploration be exhaustive. It is evident that the number of times the agent passes on cells using the optimizations is lower. This can be deduced by seeing that the heatmap concerning the optimizations contains lighter cells and has more defined edges, a symptom of a more precise and efficient exploration. In addition, one can notice the difference in the color bar of the two plots. In the first plot’s color bar the maximum value is greater than 10, meaning that the agent has stepped over more than 10 times over a certain cell. In the second color bar the maximum is slightly over 8, which means that the map has been explored more efficiently.
<br><br>



  <figure>
    <img src="https://drive.google.com/uc?id=1vYuFTjCNKDJRhD613Nv_O3BEODreXv1s" width="50%">
    <figcaption>Heatmap without optimizations</figcaption>
  </figure>

  <figure>
    <img src="https://drive.google.com/uc?id=1EK1abS9goekQpssDXse5FRNh_wQFMhua" width="50%">
    <figcaption>Heatmap with optimizations</figcaption>
  </figure>




<br>
<h4>4.3 Third Test</h4>

The last test consists of evaluating the success rate of the agent, namely the relative number of times the agent picks up the target object instead of the cursed objects. The success rate over a thousand runs is 98.9%, which means that the agent has been able to reach the correct target object in 989 of the thousand runs. This result is expected since, in our experiment, we chose a confidence for the agent of 95% before trying to get the target object of the predicted room.


<br>
<h4><b>5. CONCLUSIONS</b></h4>

This project focuses on developing intelligent algorithms for agent navigation in partially observable environments. <br>
We implemented an advanced exploration algorithm based on A* for optimal reasoning allowing us to recognize the room type and navigat in a partially observable environment reaching and picking the right target object. <br> <br>
We tested our implementation comparing it with a baseline approach.
In the first two tests we have observed that using our optimizations significantly enhances the agent's performance in exploring the map.
In the third test, the agent demonstrated a winning capability with a 98.9% success rate over 1000 runs, highlighting its effectiveness in accurately reaching the target object based on room predictions at a 95% confidence threshold. <br><br>
Finally, working in a group has allowed us to develop soft skills such as coordinating work, communicating, adaptability and flexibility which will be useful to us in the future.


<br>
<h4><b>6. APPENDIX</b></h4>

<h4>6.1 Relations with the course</h4>

The main idea of this project was to work with an exploration algorithm through a partially observable environment mixed with probabilistic reasoning.<br>
As exploration algorithm, we decided to implement a customized A* optimal for our task.<br>
The partial observability depends on the fact that the agent only can see its sourroundings and knows the shape of the room.<br>
The probabilistic factor is included in the recognition of the room by implementing the Bayes' formula

<br>
<h4>6.2 Team contributions</h4>

Each team member made significant contributions to the entire project, however:
- Aliprandi Francesco: object choice, A* implementation and its optimizations
- De Castelli Fabrizio: object recognition, code refactoring and assessments
- Di Riccio Tommaso: random room generator and map preconditioning
- Minniti Marco: object recognition and probabilistic reasoning
- Simonetti Francesco: object choice and room exploration function logic <br><br>
- All together: idea development, bug fixing, utility functions, TFFFM distance and report 


<br>
<h4>6.3 GitHub metrics</h4>
<div style="display: flex; justify-content: space-between;">
  <figure>
    <img src="https://drive.google.com/uc?id=1prP0MdiEZKpJnnI-nHEQf8QHm3M1Q4XY" width="100%">
  </figure>
</div>
